In [7]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import cv2
import random
from tqdm import tqdm

#main_folder = "C:/Users/kvvip/Udacity_MLND/Capstone/tennis/tennis_vedios/"
main_folder = "/media/karthik/Windows/Users/kvvip/Udacity_MLND/Capstone/tennis/tennis_vedios/"

def load_datasets(path):
    data = load_files(path)
    tennis_files = data['filenames']
    tennis_targets = np_utils.to_categorical(np.array(data['target']), 3)
    return tennis_files, tennis_targets

train_files, train_targets = load_datasets(main_folder+"train/")
valid_files, valid_targets = load_datasets(main_folder + "valid/")
test_files, test_targets = load_datasets(main_folder + "test/")

print(train_files.shape)

(300,)


In [2]:


def video_to_images(num, video_path, out_shape, num_iters):
    # divide the video file into "num" equal parts and randomly sample one
    # image from each part, thereby, totalling "num" images per video file
    
    video_reader = cv2.VideoCapture(video_path)

    nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    #print(nb_frames, frame_h, frame_w, video_path)
    
    frame_indexs= np.array(range(0,nb_frames+1, int(nb_frames/(num))))
    #np.append(frame_indexs,nb_frames)
    #print(frame_indexs)
    
    final_images_set = []
    for iteration in range(num_iters):
        index_start=0
        selected_frames = []
        images_out=[]
        
        for vals in frame_indexs[1:]:
            selected_frames.append(random.choice(range(index_start, vals)))
            index_start = vals

            video_reader.set(1, selected_frames[-1])
            ret, image = video_reader.read()

            #print(image.shape)
            input_image = cv2.resize(image, out_shape)
            #print(input_image.shape)
            input_image = input_image/ 255 #normalizing
            input_image=list(input_image)
            images_out.append(input_image)
        
        final_images_set.append(images_out)
    
    
    #images_out  = np.array(images_out)
    final_images_set=np.array(final_images_set)
    
    video_reader.release()
    
    return final_images_set
        
    #print(selected_frames)
    
    
    
out_images = video_to_images(5, train_files[0], (640, 360),1)
print(out_images.shape, np.expand_dims(out_images, axis=0).shape)

(1, 5, 360, 640, 3) (1, 1, 5, 360, 640, 3)


In [8]:
#potentital for parallel processing... not doing it here

def get_tensors(num_per_video, image_size_tuple, num_iterations, files):
    final_tensors= (video_to_images(num_per_video, files[0], image_size_tuple, num_iterations))
    #print(np.array(final_tensors).shape)
    #final_tensors = []
    for file in tqdm(files[1:]):
        final_tensors = np.vstack((final_tensors, video_to_images(num_per_video, file, image_size_tuple, num_iterations)))
        #final_tensors.append(video_to_images(num_per_video, file, image_size_tuple, num_iterations))

        #print(np.array(final_tensors).shape)
    return np.array(final_tensors)

def get_targets(targets, num_iters):
    final_targets =[]
    for target in tqdm(targets):
        for vals in range(num_iters):
            final_targets.append(target)
    
    return np.array(final_targets)
    

#train_tensors = get_tensors(5, (200,200), 5, train_files[:])
train_targets = get_targets(train_targets, 5)

#train_target
#train_tensors = np.array([video_to_images(5, path_ , (299,299)) for path_ in tqdm(train_files)])
valid_tensors = np.array([np.squeeze(video_to_images(5, path_ , (200,200), 1)) for path_ in tqdm(valid_files)])
test_tensors = np.array([np.squeeze(video_to_images(5, path_ , (200,200), 1)) for path_ in tqdm(test_files)])


100%|██████████| 101/101 [00:02<00:00, 41.89it/s]


In [9]:
print(train_tensors.shape, type(train_tensors))
print(train_targets.shape, type(train_targets))
print(valid_tensors.shape, type(valid_tensors))
print(test_tensors.shape, type(test_tensors))

(1500, 5, 200, 200, 3) <class 'numpy.ndarray'>
(1500, 3) <class 'numpy.ndarray'>
(100, 5, 200, 200, 3) <class 'numpy.ndarray'>
(101, 5, 200, 200, 3) <class 'numpy.ndarray'>


In [22]:
from keras.layers import Conv3D, MaxPooling3D, GlobalAveragePooling3D,MaxPooling2D, Conv2D
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Input, AveragePooling3D, GlobalAveragePooling2D
from keras.models import Sequential
"""
model = Sequential()

### TODO: Define your architecture.
model.add(Conv2D(8, (2,2), strides=2, padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(16, (2,2), strides=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, (2,2), strides=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
#model.add(Dense(250, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(133, activation='softmax'))

model.summary()

"""

model = Sequential()
model.add(Conv3D(8, (2,2,2), strides=(1,2,2), padding='same', activation='relu', input_shape=(5, 200, 200, 3)))
model.add(MaxPooling3D(pool_size=(1,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))

model.add(Conv3D(16, (2,2,2), strides=(1,2,2), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(1,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))

model.add(Conv3D(32, (2,2,2), strides=(1,2,2), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))

model.add(Conv3D(64, (2,2,2), strides=(1,2,2), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_last"))


model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()


"""
input_video = Input(shape=(5, 180, 320, 3))

#layer1
x=Conv3D(8, (2,2,2), strides=(1,2,2), padding='same', name='conv_1', use_bias=True)(input_video)
#x = BatchNormalization(name='norm_1')(x)
#x = LeakyReLU(alpha=0.1)(x)
x = AveragePooling3D(pool_size=(1, 2, 2))(x)

print(x.shape)
"""



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 5, 100, 100, 8)    200       
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 5, 50, 50, 8)      0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 5, 25, 25, 16)     1040      
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 5, 12, 12, 16)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 5, 6, 6, 32)       4128      
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 2, 3, 3, 32)       0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 2, 2, 2, 64)       16448     
__________

"\ninput_video = Input(shape=(5, 180, 320, 3))\n\n#layer1\nx=Conv3D(8, (2,2,2), strides=(1,2,2), padding='same', name='conv_1', use_bias=True)(input_video)\n#x = BatchNormalization(name='norm_1')(x)\n#x = LeakyReLU(alpha=0.1)(x)\nx = AveragePooling3D(pool_size=(1, 2, 2))(x)\n\nprint(x.shape)\n"

In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
from keras.callbacks import ModelCheckpoint  

### specify the number of epochs that you would like to use to train the model.

epochs = 20


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

Train on 1500 samples, validate on 100 samples
Epoch 1/30
Epoch 00001: val_loss improved from inf to 1.06157, saving model to saved_models/weights.best.from_scratch.hdf5
 - 5s - loss: 1.0788 - acc: 0.4140 - val_loss: 1.0616 - val_acc: 0.4200
Epoch 2/30
Epoch 00002: val_loss improved from 1.06157 to 1.05087, saving model to saved_models/weights.best.from_scratch.hdf5
 - 4s - loss: 1.0478 - acc: 0.4387 - val_loss: 1.0509 - val_acc: 0.4200
Epoch 3/30
Epoch 00003: val_loss improved from 1.05087 to 0.91229, saving model to saved_models/weights.best.from_scratch.hdf5
 - 4s - loss: 0.9082 - acc: 0.5473 - val_loss: 0.9123 - val_acc: 0.5500
Epoch 4/30
Epoch 00004: val_loss improved from 0.91229 to 0.90216, saving model to saved_models/weights.best.from_scratch.hdf5
 - 4s - loss: 0.7325 - acc: 0.6640 - val_loss: 0.9022 - val_acc: 0.5300
Epoch 5/30
Epoch 00005: val_loss improved from 0.90216 to 0.88461, saving model to saved_models/weights.best.from_scratch.hdf5
 - 5s - loss: 0.6700 - acc: 0.6840

In [25]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [26]:
# get index of predicted dog breed for each image in test set
tennis_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(tennis_predictions)==np.argmax(test_targets, axis=1))/len(tennis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 69.3069%


In [27]:
from keras.applications.inception_v3 import InceptionV3

input_tensor = Input(shape=(200, 200, 3))
# create the base pre-trained model
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)


In [28]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 99, 99, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 99, 99, 32)   96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 99, 99, 32)   0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [29]:
def bottleneck_feature_extractor(inp_tensors, model_used, length=1500, num_images=5):
    
    #train tensors are of shape (300, 5, 299,299,3)
    #copy_inp_tensors = inp_tensors.copy()
    #copy_inp_tensors = np.array(np.squeeze(np.split(inp_tensors, num_images, axis=1)))
    copy_inp_tensors= np.reshape(inp_tensors, (num_images, length, 200, 200, 3))
    #copy_inp_tensors of shape (5, 300, 299,299,3)
    bottleneck_features=[]
    for frame_images in tqdm(copy_inp_tensors):
        bottleneck_features.append(model_used.predict(frame_images))
        
    #BNeck_tensors1 = np.squeeze(np.split(np.array(bottleneck_features), length, axis=1))
    bottleneck_features = np.array(bottleneck_features)
    a,b,c,d,e = bottleneck_features.shape
    BNeck_tensors = np.reshape(bottleneck_features, (b, a, c, d, e))
    #print(BNeck_tensors == BNeck_tensors1)
    return BNeck_tensors


train_BNeck_tensors = bottleneck_feature_extractor(train_tensors, base_model)
valid_BNeck_tensors = bottleneck_feature_extractor(valid_tensors, base_model, len(valid_tensors),5)
test_BNeck_tensors = bottleneck_feature_extractor(test_tensors, base_model, len(test_tensors), 5)

print(train_BNeck_tensors.shape, valid_BNeck_tensors.shape, test_BNeck_tensors.shape)
   
        
    
    

100%|██████████| 5/5 [00:01<00:00,  2.98it/s]

(1500, 5, 4, 4, 2048) (100, 5, 4, 4, 2048) (101, 5, 4, 4, 2048)


In [60]:
BNeck_model = Sequential()
BNeck_model.add(Conv3D(256, (2,2,2), strides=(2,2,2), padding='same', activation='relu', input_shape=(5, 4, 4, 2048), data_format="channels_first"))
#BNeck_model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_first"))
model.add(Dropout(0.8))

BNeck_model.add(Conv3D(256, (2,2,2), strides=(2,2,2), padding='same', activation='relu', data_format="channels_first"))
#BNeck_model.add(MaxPooling3D(pool_size=(1,2,2), strides=None, padding='valid', data_format="channels_first"))
model.add(Dropout(0.95))

#BNeck_model.add(Conv3D(32, (2,2,2), strides=(2,2,2), padding='same', activation='relu'))
#BNeck_model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))

#BNeck_model.add(Conv3D(64, (2,2,2), strides=(1,2,2), padding='same', activation='relu'))
#BNeck_model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_last"))


BNeck_model.add(Flatten())
#BNeck_model.add(Dense(5, activation='relu'))
#model.add(Dropout(0.3))
BNeck_model.add(Dense(3, activation='softmax'))

BNeck_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_37 (Conv3D)           (None, 256, 2, 2, 1024)   10496     
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 256, 1, 1, 512)    524544    
_________________________________________________________________
flatten_17 (Flatten)         (None, 131072)            0         
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 393219    
Total params: 928,259
Trainable params: 928,259
Non-trainable params: 0
_________________________________________________________________


In [61]:
BNeck_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### specify the number of epochs that you would like to use to train the model.

epochs = 10


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_InceptionV3.hdf5', 
                               verbose=1, save_best_only=True)

BNeck_model.fit(train_BNeck_tensors, train_targets, 
          validation_data=(valid_BNeck_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

Train on 1500 samples, validate on 100 samples
Epoch 1/10
Epoch 00001: val_loss improved from inf to 0.75870, saving model to saved_models/weights.best.from_InceptionV3.hdf5
 - 4s - loss: 0.7378 - acc: 0.6853 - val_loss: 0.7587 - val_acc: 0.6300
Epoch 2/10
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.1235 - acc: 0.9607 - val_loss: 0.7759 - val_acc: 0.7300
Epoch 3/10
Epoch 00003: val_loss improved from 0.75870 to 0.70097, saving model to saved_models/weights.best.from_InceptionV3.hdf5
 - 3s - loss: 0.0228 - acc: 0.9927 - val_loss: 0.7010 - val_acc: 0.7300
Epoch 4/10
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.0030 - acc: 1.0000 - val_loss: 0.9568 - val_acc: 0.7500
Epoch 5/10
Epoch 00005: val_loss did not improve
 - 3s - loss: 6.7052e-04 - acc: 1.0000 - val_loss: 1.0380 - val_acc: 0.7700
Epoch 6/10
Epoch 00006: val_loss did not improve
 - 3s - loss: 2.8013e-04 - acc: 1.0000 - val_loss: 1.0856 - val_acc: 0.7600
Epoch 7/10
Epoch 00007: val_loss did not improve
 - 3s - lo

In [59]:
# get index of predicted dog breed for each image in test set
tennis_predictions = [np.argmax(BNeck_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_BNeck_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(tennis_predictions)==np.argmax(test_targets, axis=1))/len(tennis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)



Test accuracy: 79.2079%


In [129]:
def bottleneck_feature_extractor_avg(inp_tensors, model_used, length=300, num_images=5):
    
    #train tensors are of shape (300, 5, 299,299,3)
    #copy_inp_tensors = inp_tensors.copy()
    #copy_inp_tensors = np.array(np.squeeze(np.split(inp_tensors, num_images, axis=1)))
    copy_inp_tensors= np.reshape(inp_tensors, (num_images, length, 299, 299, 3))
    #copy_inp_tensors of shape (5, 300, 299,299,3)
    bottleneck_features=[]
    for frame_images in tqdm(copy_inp_tensors):
        model_used.add(GlobalAveragePooling2D())#(model_used)
        bottleneck_features.append(model_used.predict(frame_images))
        
    #BNeck_tensors1 = np.squeeze(np.split(np.array(bottleneck_features), length, axis=1))
    bottleneck_features = np.array(bottleneck_features)
    a,b,c,d,e = bottleneck_features.shape
    BNeck_tensors = np.reshape(bottleneck_features, (b, a, c, d, e))
    #print(BNeck_tensors == BNeck_tensors1)
    return BNeck_tensors


train_BNeck_tensors = bottleneck_feature_extractor_avg(train_tensors, base_model)
valid_BNeck_tensors = bottleneck_feature_extractor_avg(valid_tensors, base_model, len(valid_tensors),5)
test_BNeck_tensors = bottleneck_feature_extractor_avg(test_tensors, base_model, len(test_tensors), 5)

print(train_BNeck_tensors.shape, valid_BNeck_tensors.shape, test_BNeck_tensors.shape)
   

  0%|          | 0/5 [00:00<?, ?it/s]


AttributeError: 'Model' object has no attribute 'add'